# Seq2Seq with Attention

In [57]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [28]:
source = open('data/commands_source_en.txt', encoding='UTF-8').read().strip().split('\n')  
target = open('data/commands_target_en.txt', encoding='UTF-8').read().strip().split('\n')  

In [31]:
data = pd.DataFrame(list(zip(source, target)), columns=["src", "tgt"])
data.head()

,src,tgt
0,what's the temperature like,what's the temperature like
1,can you tell me how warm it will get on 12.10,can you tell me how warm it will get on twelft...
2,how hot will it get on 20.7,how hot will it get on twentieth july
3,I want to know how warm it is in Tremosine fro...,I want to know how warm it is in tremosine fro...
4,how is the temperature between 13:28 and 23:17...,how is the temperature between thirteen twenty...


In [58]:
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [65]:
# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([!?])", r" \1", s)
#     s = re.sub(r"[^a-zA-Z!?]+", r" ", s)
    return s

In [99]:
def readLangs(lang1, lang2):
    print("Reading lines...")

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s), normalizeString(t) ]for s, t in list(zip(target, source))] 

    input_lang = Lang(lang1)
    output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [100]:
def prepareData(lang1, lang2):
    input_lang, output_lang, pairs = readLangs(lang1, lang2)
    print("Read %s sentence pairs" % len(pairs))
#     pairs = filterPairs(pairs)
#     print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('src', 'tgt')
print(random.choice(pairs))

Reading lines...
Read 7000 sentence pairs
Counting words...
Counted words:
src 2032
tgt 2366
['i wanna switch to radio channel twenty two', 'i wanna switch to radio channel 22']


In [93]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [68]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [70]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=50):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [71]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [73]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=50):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [74]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [75]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [77]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [79]:
def evaluate(encoder, decoder, sentence, max_length=50):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [80]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [101]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

trainIters(encoder1, attn_decoder1, 72000, print_every=3000)

1m 18s (- 30m 6s) (3000 4%) 3.1985
2m 33s (- 28m 13s) (6000 8%) 1.5453
3m 50s (- 26m 50s) (9000 12%) 1.1344
5m 7s (- 25m 36s) (12000 16%) 0.9387
6m 25s (- 24m 24s) (15000 20%) 0.8091
7m 43s (- 23m 9s) (18000 25%) 0.6900
9m 0s (- 21m 51s) (21000 29%) 0.6099
10m 18s (- 20m 36s) (24000 33%) 0.5150
11m 35s (- 19m 19s) (27000 37%) 0.4803
12m 53s (- 18m 2s) (30000 41%) 0.4315
14m 11s (- 16m 45s) (33000 45%) 0.3892
15m 28s (- 15m 28s) (36000 50%) 0.3629
16m 46s (- 14m 12s) (39000 54%) 0.3198
18m 5s (- 12m 55s) (42000 58%) 0.2562
19m 22s (- 11m 37s) (45000 62%) 0.2559
20m 40s (- 10m 20s) (48000 66%) 0.2421
21m 59s (- 9m 3s) (51000 70%) 0.2386
23m 17s (- 7m 45s) (54000 75%) 0.1850
24m 35s (- 6m 28s) (57000 79%) 0.1524
25m 53s (- 5m 10s) (60000 83%) 0.1441
27m 10s (- 3m 52s) (63000 87%) 0.1389
28m 29s (- 2m 35s) (66000 91%) 0.1134
29m 46s (- 1m 17s) (69000 95%) 0.1000
31m 4s (- 0m 0s) (72000 100%) 0.0860


In [102]:
evaluateRandomly(encoder1, attn_decoder1)

> how is the weather forecast for tirol
= how is the weather forecast for tirol
< how is the weather forecast for tirol <EOS>

> i'd like to watch thirteen rue
= i'd like to watch 13 rue
< i'd like to watch 13 rue <EOS>

> i want to know more about jessica chastain
= i want to know more about jessica chastain
< i want to know more about jessica chastain <EOS>

> switch to a different radio station
= switch to a different radio station
< switch to a different radio station <EOS>

> please turn device plant light on
= please turn device plant light on
< please turn device plant light on <EOS>

> will it rain on twenty first september
= will it rain on 21.9
< will it rain on 21.9 <EOS>

> turn on all philips in the home office
= turn on all philips in the home office
< turn on all philips in the home office <EOS>

> do i need an umbrella in schlanders on seventh april
= do i need an umbrella in schlanders on 7.4
< do i need an umbrella in schlanders on 7.4 <EOS>

> channel change
= channe

In [124]:
evaluateRandomly(encoder1, attn_decoder1)

> i want you to switch the device tv sony sonos on
= i want you to switch the device tv sony sonos on
< i want you to switch the device tv sony sonos on <EOS>

> will it be rainy on sixth june
= will it be rainy on 6.6
< will it be rainy on 6.6 <EOS>

> please activate my philips hue mystrom wifi switch in the children's room
= please activate my philips hue mystrom wifi switch in the children's room
< please activate my philips hue mystrom wifi switch in the children's room <EOS>

> turn on my mystrom
= turn on my mystrom
< turn on my mystrom <EOS>

> show me skn saint polten
= show me skn st. polten
< show me viceroy's st. polten <EOS>

> put on classica please
= put on classica please
< put on classica please <EOS>

> i want to listen to something else
= i want to listen to something else
< i want to listen to something else <EOS>

> can you give me the weather report
= can you give me the weather report
< can you give me the weather report <EOS>

> weather update please
= weather u

In [149]:
def showAttention(input_sentence, output_words, attentions):
    # Set up figure with colorbar
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(attentions.numpy(), cmap='bone')
    fig.colorbar(cax)

    # Set up axes
    ax.set_xticklabels([''] + input_sentence.split(' ') +
                       ['<EOS>'], rotation=90)
    ax.set_yticklabels([''] + output_words)

    # Show label at every tick
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()


def evaluateAndShowAttention(input_sentence):
    output_words, attentions = evaluate(
        encoder1, attn_decoder1, input_sentence)
    print('input =', input_sentence)
    print('output =', ' '.join(output_words))
#     showAttention(input_sentence, output_words, attentions)
    
evaluateAndShowAttention("change to radio channel sixty four please")

input = change to radio channel sixty four please
output = change to radio channel 64 please <EOS>


## issue1: unkown mapping -> larger dataset

In [151]:
evaluateAndShowAttention("change to radio channel sixty two please")

input = change to radio channel sixty two please
output = change to radio channel 42 please <EOS>


## issue2: unkown words -> GloVe

In [168]:
evaluateAndShowAttention("i'd like to watch lord of the ring")

KeyError: 'ring'